In [16]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np

torch.manual_seed(2019)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [19]:
train_data = torchvision.datasets.CIFAR100('./data/', train=True, transform= trans, download=True)
test_data = torchvision.datasets.CIFAR100('./data/', train=False, transform= trans, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
batch_size = 64

conv_channel_1 = 64
conv_channel_2 = 64
conv_channel_3 = 128
conv_channel_4 = 128
conv_channel_5 = 256

hidden1_size = 1024
#hidden2_size = 300
hidden2_size = 1024
out_size = 100
epoch = 50

In [0]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=True)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [0]:
loss = nn.CrossEntropyLoss()

In [0]:
import torch.optim as optim
from torch.autograd import Variable

def fit(epoch, data_loader, model, opt, phase):
    if phase=='train':
        volatile = False
        model.train()
    else:
        volatile = True
        model.eval()       
        
    learning_loss = 0.
    learning_correct = 0
    
    for i, (inputs, labels) in enumerate(data_loader):
        if phase == 'train':
            opt.zero_grad()
            
        inputs = Variable(inputs, volatile).cuda()
        labels = Variable(labels).cuda()
        outputs = model(inputs)
        
        loss_value = loss(outputs, labels)
        learning_loss += loss_value.item() * inputs.shape[0]
        
        if phase == 'train':
            loss_value.backward()
            opt.step()
        
        _, preds = torch.max(outputs.data, 1)
        learning_correct += (preds == labels).sum().item()
        
    epoch_loss = learning_loss /len(data_loader.dataset)
    epoch_correct = learning_correct / len(data_loader.dataset) * 100.
    
    print(f'epoch = {epoch} -> {phase:{5}} / loss = {epoch_loss:{8}.{3}}, correct = {epoch_correct:{8}.{4}}%')

In [0]:
class commonLinear(nn.Module):
    def __init__(self, nin):
        super(commonLinear, self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(nin, hidden2_size),
            nn.ReLU(),
#             nn.Linear(hidden1_size, hidden2_size),
#             nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden2_size, out_size),
            nn.ReLU()
        )
        
    def forward(self,x):
        x=self.fc(x)
        return F.log_softmax(x, dim=1)

In [0]:
class normalCNN(nn.Module):
    def __init__(self):
        super(normalCNN,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_2, conv_channel_3, 3, padding=1),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.Conv2d(conv_channel_3, conv_channel_4, 3, padding=1),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_4, conv_channel_5, 3, padding=1),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [36]:
cnn1 = normalCNN().cuda()
#opt1 = optim.Adadelta(cnn1.parameters(), rho=0.95)
opt1 = optim.SGD(cnn1.parameters(), lr=0.01, momentum=0.9)
print(cnn1)

normalCNN(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(

In [37]:
for ep in range(epoch):
    t= time.time()
    fit(ep, train_loader, cnn1, opt1, 'train')
    fit(ep, test_loader, cnn1, opt1, 'eval')
    print('%.4f'%(time.time()-t))
    print()

epoch = 0 -> train / loss =     3.59, correct =    16.05%
epoch = 0 -> eval  / loss =     2.79, correct =    29.34%
16.7973

epoch = 1 -> train / loss =     2.65, correct =    31.96%
epoch = 1 -> eval  / loss =     2.37, correct =    38.04%
16.7815

epoch = 2 -> train / loss =     2.24, correct =    40.75%
epoch = 2 -> eval  / loss =     2.06, correct =    45.39%
16.8956

epoch = 3 -> train / loss =     1.95, correct =    47.21%
epoch = 3 -> eval  / loss =     1.84, correct =    50.59%
16.8733

epoch = 4 -> train / loss =     1.72, correct =    52.46%
epoch = 4 -> eval  / loss =      2.0, correct =    48.11%
17.0561

epoch = 5 -> train / loss =     1.54, correct =    56.81%
epoch = 5 -> eval  / loss =     1.62, correct =    55.62%
17.1037

epoch = 6 -> train / loss =     1.37, correct =    60.85%
epoch = 6 -> eval  / loss =     1.59, correct =    56.56%
16.9674

epoch = 7 -> train / loss =     1.23, correct =    64.35%
epoch = 7 -> eval  / loss =     1.57, correct =    56.83%
17.0352



In [0]:
class depthwiseCNN(nn.Module):
    def __init__(self,nin,nout):
        super(depthwiseCNN,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(nin,nin,3,groups=nin, padding=1),
            nn.BatchNorm2d(nin),
            nn.ReLU(),
            nn.Conv2d(nin,nout,1),
            nn.BatchNorm2d(nout),
            nn.ReLU()
        )

    def forward(self,x):
        return self.model(x)
        
class separableCNN(nn.Module):
    def __init__(self):
        super(separableCNN,self).__init__()
        self.conv = nn.Sequential(
            depthwiseCNN(3,conv_channel_1),
            depthwiseCNN(conv_channel_1,conv_channel_2),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_2,conv_channel_3),
            depthwiseCNN(conv_channel_3,conv_channel_4),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_4,conv_channel_5),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [39]:
cnn2 = separableCNN().cuda()
#opt2 = optim.Adadelta(cnn2.parameters(), rho=0.95)
opt2 = optim.SGD(cnn2.parameters(), lr=0.01, momentum=0.9)
print(cnn2)

separableCNN(
  (conv): Sequential(
    (0): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (1): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [40]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn2, opt2, 'train')
    fit(ep, test_loader, cnn2, opt2, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     3.93, correct =    11.57%
epoch = 0 -> eval  / loss =     3.17, correct =    23.29%
18.1486

epoch = 1 -> train / loss =     2.96, correct =    26.19%
epoch = 1 -> eval  / loss =     2.57, correct =     33.8%
18.0995

epoch = 2 -> train / loss =     2.52, correct =    34.94%
epoch = 2 -> eval  / loss =     2.23, correct =    40.85%
18.4070

epoch = 3 -> train / loss =     2.24, correct =    40.52%
epoch = 3 -> eval  / loss =     2.15, correct =    42.52%
18.1394

epoch = 4 -> train / loss =     2.03, correct =    45.54%
epoch = 4 -> eval  / loss =     2.04, correct =    45.67%
18.2060

epoch = 5 -> train / loss =     1.84, correct =    49.41%
epoch = 5 -> eval  / loss =     1.86, correct =    49.23%
18.3595

epoch = 6 -> train / loss =     1.71, correct =    52.84%
epoch = 6 -> eval  / loss =      1.8, correct =    51.79%
18.3463

epoch = 7 -> train / loss =     1.57, correct =    55.98%
epoch = 7 -> eval  / loss =     1.85, correct =    50.51%
18.4323



In [0]:
class inception(nn.Module):
    def __init__(self, nin, nout):
        super(inception,self).__init__()
        self.model1x1 = nn.Sequential(
            nn.Conv2d(nin, int(nout/4), 1)
        )
        self.model3x3 = nn.Sequential(
            nn.Conv2d(nin,int(nout/8),1),
            nn.Conv2d(int(nout/8),int(nout/4),3, padding=1)
        )
        self.model5x5 = nn.Sequential(
            nn.Conv2d(nin,int(nout/16),1),
            nn.Conv2d(int(nout/16),int(nout/8),3,padding=1),
            nn.Conv2d(int(nout/8),int(nout/4),3,padding=1)
        )
        self.modelmax = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(nin,int(nout/4),1)
        )
        
    def forward(self, x):
        part1x1=self.model1x1(x)
        part3x3=self.model3x3(x)
        part5x5=self.model5x5(x)
        partmax=self.modelmax(x)
        outputs= [part1x1, part3x3, part5x5, partmax]
        return torch.cat(outputs,1)
    
class inceptionCNN(nn.Module):
    def __init__(self):
        super(inceptionCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_2, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            inception(conv_channel_2, conv_channel_3),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            inception(conv_channel_3, conv_channel_4),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            inception(conv_channel_4, conv_channel_5),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc=commonLinear(conv_channel_5*4*4)
    
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1, conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [42]:
cnn3 = inceptionCNN().cuda()
#opt3 = optim.Adadelta(cnn3.parameters(), rho=0.95)
opt3 = optim.SGD(cnn3.parameters(), lr=0.01, momentum=0.9)
print(cnn3)

inceptionCNN(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): inception(
      (model1x1): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (model3x3): Sequential(
        (0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (model5x5): Sequential(
        (

In [43]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn3, opt3, 'train')
    fit(ep, test_loader, cnn3, opt3, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     3.59, correct =    15.88%
epoch = 0 -> eval  / loss =     2.75, correct =    29.42%
21.8676

epoch = 1 -> train / loss =     2.66, correct =    31.74%
epoch = 1 -> eval  / loss =     2.39, correct =    37.41%
21.8273

epoch = 2 -> train / loss =     2.25, correct =    40.24%
epoch = 2 -> eval  / loss =     2.06, correct =    45.76%
21.7285

epoch = 3 -> train / loss =     1.97, correct =    46.49%
epoch = 3 -> eval  / loss =     1.91, correct =    49.35%
21.8206

epoch = 4 -> train / loss =     1.77, correct =    51.09%
epoch = 4 -> eval  / loss =     1.84, correct =    50.37%
21.7078

epoch = 5 -> train / loss =     1.59, correct =    55.27%
epoch = 5 -> eval  / loss =      1.7, correct =    53.31%
21.8872

epoch = 6 -> train / loss =     1.45, correct =    58.77%
epoch = 6 -> eval  / loss =     1.66, correct =    54.76%
21.6896

epoch = 7 -> train / loss =     1.31, correct =    62.28%
epoch = 7 -> eval  / loss =     1.58, correct =    56.67%
21.7583

